In [62]:
import numpy as np
import pandas as pd
import os

In [104]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
from etl_helpers import *
from model_helpers import *

## Read in player results and salaries

In [79]:
df_dict, df_salaries, df_points = import_data()

In [80]:
df_dict.head()

,TeamID,Place,Team,Identifier,Abbreviation1,Abbreviation2
0,0,Arizona,Cardinals,Arizona,ARI,ari
1,1,Atlanta,Falcons,Atlanta,ATL,atl
2,2,Baltimore,Ravens,Baltimore,BAL,bal
3,3,Buffalo,Bills,Buffalo,BUF,buf
4,4,Carolina,Panthers,Carolina,CAR,car


In [81]:
df_salaries.head()

,Position,Name,Salary,GameInfo,AvgPointsPerGame,teamAbbrev
0,WR,Antonio Brown,9300,PIT@BAL 01:00PM ET,24.467,PIT
1,WR,Odell Beckham Jr.,8900,NYG@TB 04:05PM ET,18.250,NYG
2,WR,Julio Jones,8800,BUF@ATL 01:00PM ET,15.167,ATL
3,RB,Le'Veon Bell,8700,PIT@BAL 01:00PM ET,14.200,PIT
4,WR,A.J. Green,8600,CIN@CLE 01:00PM ET,18.067,CIN


In [82]:
df_points.head()

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,DK points,DK salary
0,1,2017,1254,"Smith, Alex",QB,kan,a,nwe,34.02,5400.0
1,1,2017,1344,"Bradford, Sam",QB,min,h,nor,28.54,5900.0
2,1,2017,1340,"Stafford, Matthew",QB,det,h,ari,28.08,6100.0
3,1,2017,1514,"Wentz, Carson",QB,phi,a,was,23.88,5300.0
4,1,2017,1488,"Siemian, Trevor",QB,den,h,lac,23.66,5000.0


## Clean and merge data

In [87]:
df = clean_and_merge(df_dict=df_dict, df_salaries=df_salaries, df_points=df_points)

Entries in salary dataframe: 570
Entries in points dataframe: 1246
Merged 1121 player entries
Merged 94 team entries
Merged 1215 entries total


In [89]:
df.head()

,TeamID,index_s,Position,Name_s,Salary,GameInfo,AvgPointsPerGame,Place_s,Team_s,Identifier_s,...,h/a,Oppt,DK points,DK salary,Place,Team,Identifier,Abbreviation1,Firstname,Lastname
1,0,ARI,RB,David Johnson,6300,SF@ARI 04:05PM ET,14.100,Arizona,Cardinals,Arizona,...,a,det,14.10,9400.0,Arizona,Cardinals,Arizona,ARI,DAVID,JOHNSON
35,0,ARI,QB,Carson Palmer,6100,SF@ARI 04:05PM ET,18.847,Arizona,Cardinals,Arizona,...,a,det,11.96,6000.0,Arizona,Cardinals,Arizona,ARI,CARSON,PALMER
47,0,ARI,QB,Carson Palmer,6100,SF@ARI 04:05PM ET,18.847,Arizona,Cardinals,Arizona,...,a,ind,19.88,6000.0,Arizona,Cardinals,Arizona,ARI,CARSON,PALMER
58,0,ARI,QB,Carson Palmer,6100,SF@ARI 04:05PM ET,18.847,Arizona,Cardinals,Arizona,...,h,dal,24.70,6000.0,Arizona,Cardinals,Arizona,ARI,CARSON,PALMER
75,0,ARI,WR,Larry Fitzgerald,6100,SF@ARI 04:05PM ET,18.467,Arizona,Cardinals,Arizona,...,a,det,13.40,5900.0,Arizona,Cardinals,Arizona,ARI,LARRY,FITZGERALD


In [88]:
df.dtypes

TeamID                int64
index_s              object
Position             object
Name_s               object
Salary                int64
GameInfo             object
AvgPointsPerGame    float64
Place_s              object
Team_s               object
Identifier_s         object
Abbreviation2        object
Firstname_s          object
Lastname_s           object
index                object
Week                  int64
Year                  int64
GID                   int64
Name                 object
Pos                  object
h/a                  object
Oppt                 object
DK points           float64
DK salary           float64
Place                object
Team                 object
Identifier           object
Abbreviation1        object
Firstname            object
Lastname             object
dtype: object

In [90]:
df.describe()

,TeamID,Salary,AvgPointsPerGame,Week,Year,GID,DK points,DK salary
count,1215.000000,1215.000000,1215.000000,1215.000000,1215.0,1215.000000,1215.000000,1215.000000
mean,15.241152,3986.337449,7.027020,2.028807,2017.0,4514.488889,7.046321,3907.654321
std,9.246592,1477.333745,6.346941,0.814809,0.0,1470.688864,7.950869,1467.444297
min,0.000000,2100.000000,-0.300000,1.000000,2017.0,1131.000000,-4.000000,0.000000
25%,7.000000,3000.000000,1.567000,1.000000,2017.0,3627.000000,0.100000,3000.000000
50%,15.000000,3500.000000,5.667000,2.000000,2017.0,4706.000000,4.200000,3500.000000
75%,23.000000,4700.000000,10.667000,3.000000,2017.0,5456.000000,11.000000,4600.000000
max,31.000000,9300.000000,34.600000,3.000000,2017.0,7032.000000,49.600000,9800.000000


## Create features

## Pick team

In [95]:
weeks = df.Week.unique()

In [100]:
team = sample_team(df=df.loc[df.Week == weeks.max()], seed=0)

In [101]:
team

,GID,DK points,Salary
0,1336,30.18,4800
1,5507,0.00,3000
2,2647,4.10,3800
3,5260,11.00,4600
4,5186,7.20,4600
5,5351,0.00,3300
6,4605,2.00,2500
7,5239,3.10,3000
8,7012,6.00,3600


In [109]:
get_performance(team, point_col='DK points')

63.580000000000005

In [106]:
get_salary(team)

33200

## Draw random teams

In [112]:
weeks = df.Week.unique()

In [114]:
bestlist = multi_draw(df=df.loc[df.Week == weeks.max()], n=100, point_col='DK points')

In [115]:
np.shape(bestlist)

(99, 2)

In [117]:
bestteam = sample_team(df=df, seed=bestlist[0][1])

In [120]:
get_performance(bestteam)

43.4

In [119]:
get_salary(bestteam)

37100

In [185]:
df.loc[df.GID.isin(bestteam.GID.values), ['GID', 'Team', 'Name', 'Position', 'Salary', 'DK points']]

,GID,Team,Name,Position,Salary,DK points
266,5159,Falcons,"Sanu, Mohamed",WR,4500,12.80
1033,5447,Bears,"Howard, Jordan",RB,4900,35.40
2544,4538,Packers,"Kendricks, Lance",TE,2500,13.20
2569,1518,Texans,"Watson, Deshaun",QB,4700,25.14
3502,5252,Rams,"Watkins, Sammy",WR,5500,31.60
4072,5255,Patriots,"Cooks, Brandin",WR,7300,35.10
6331,2915,Redskins,"Thompson, Chris",RB,4500,33.80
26,7024,Steelers,Pittsburgh,DST,3400,6.00
